# Task 1: Retrieval System — 6 pts

In [9]:
%pip install -r ../requirements.txt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 18.2 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependenc

In [21]:
# --- Importar librerías necesarias ---
import feedparser        # Para leer el RSS de RPP
import pandas as pd      # Para manejar los datos en DataFrame
import tiktoken          # Para contar tokens
import chromadb          # Base de datos vectorial
from chromadb.config import Settings
import tiktoken
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import pandas as pd
import sentencepiece


In [ ]:
# --- Leer el RSS de RPP ---
RSS_URL = "https://rpp.pe/rss"
feed = feedparser.parse(RSS_URL)
items = feed["items"][:50] # Limitar a las primeras 50 noticias
# --- Extraer campos relevantes ---
news = []
for it in items:
    news.append({
        "title": it.get("title", ""),             # Título de la noticia
        "description": it.get("summary", ""),     # Resumen o descripción
        "link": it.get("link", ""),               # Enlace original
        "published": it.get("published", "")      # Fecha
    })
# --- Crear DataFrame ---
df_news = pd.DataFrame(news)
# --- Mostrar primeras 5 filas ---
print("Número de noticias descargadas:", len(df_news))
df_news.head()


Número de noticias descargadas: 50


,title,description,link,published
0,Universitario vs. Ayacucho FC EN VIVO: juegan ...,Universitario sale con todo de local frente a ...,https://rpp.pe/futbol/descentralizado/universi...,"Mon, 20 Oct 2025 16:45:33 -0500"
1,Cúal fue el último temblor en México hoy 20 de...,Cuál es el ultimo temblor en México y CDMX reg...,https://rpp.pe/mundo/mexico/cual-fue-el-ultimo...,"Sun, 19 Oct 2025 06:45:35 -0500"
2,"Hoy, Universitario vs Ayacucho FC VIVO: ¿a qué...",Universitario recibe a Ayacucho FC por la fech...,https://rpp.pe/futbol/descentralizado/universi...,"Mon, 20 Oct 2025 16:45:07 -0500"
3,"Lula da Silva presume que Brasil ""no bajó la c...",Lula defendió la firmeza de Brasil frente a lo...,https://rpp.pe/mundo/actualidad/lula-da-silva-...,"Mon, 20 Oct 2025 16:43:33 -0500"
4,"Partidos de hoy, lunes 20 de octubre del 2025:...",Fútbol EN VIVO | Horarios y canales de TV para...,https://rpp.pe/futbol/futbol-mundial/partidos-...,"Mon, 20 Oct 2025 16:30:54 -0500"


In [12]:
enc = tiktoken.get_encoding("cl100k_base") # Creamos el codificador del modelo cl100k_base (el mismo de GPT-3.5/4)

def count_tokens(text: str) -> int:
    """Devuelve cuántos tokens hay en un texto."""
    return len(enc.encode(text))

df_news["n_tokens"] = df_news["description"].apply(count_tokens) # Aplicamos la función sobre la columna "description"

print("Promedio de tokens por descripción:", df_news["n_tokens"].mean()) # Vemos las más largas y el promedio
df_news.sort_values("n_tokens", ascending=False).head(5)

Promedio de tokens por descripción: 48.3


,title,description,link,published,n_tokens
31,"Siete minutos, cuatro ladrones y joyas de valo...",Poco a poco se conocen nuevos detalles del hur...,https://rpp.pe/mundo/europa/siete-minutos-cuat...,"Mon, 20 Oct 2025 15:46:00 -0500",118
48,Caída en la nube de Amazon causa fallos en sit...,Usuarios de Internet en todo el mundo enfrenta...,https://rpp.pe/tecnologia/mas-tecnologia/caida...,"Mon, 20 Oct 2025 13:14:30 -0500",107
8,Señor de Luren: así es la salida de la imagen ...,Miles de fieles acompañarán al patrono de Ica ...,https://rpp.pe/peru/actualidad/senor-de-luren-...,"Mon, 20 Oct 2025 12:00:35 -0500",96
34,El “queso artesanal” de Laive resultó ser indu...,Indecopi sancionó a Laive S.A. con 20 UIT (S/ ...,https://rpp.pe/economia/economia/laive-es-mult...,"Mon, 20 Oct 2025 13:00:32 -0500",95
44,¡Tendencia a la baja! Dólar sigue cayendo y ci...,El dólar continúa su tendencia bajista en el P...,https://rpp.pe/economia/economia/dolar-en-peru...,"Mon, 20 Oct 2025 13:55:49 -0500",89


In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2") # Cargar el modelo
df_news["embedding"] = df_news["description"].apply(lambda x: model.encode(x)) # Aplicar a cada descripción
print("Dimensión del embedding:", len(df_news["embedding"][0])) # Ver la dimensión del embedding
df_news.head(2)

/opt/anaconda3/envs/LLMs/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dimensión del embedding: 384


,title,description,link,published,n_tokens,embedding
0,Universitario vs. Ayacucho FC EN VIVO: juegan ...,Universitario sale con todo de local frente a ...,https://rpp.pe/futbol/descentralizado/universi...,"Mon, 20 Oct 2025 16:45:33 -0500",32,"[-0.007385407, 0.0169205, -0.05339155, -0.0102..."
1,Cúal fue el último temblor en México hoy 20 de...,Cuál es el ultimo temblor en México y CDMX reg...,https://rpp.pe/mundo/mexico/cual-fue-el-ultimo...,"Sun, 19 Oct 2025 06:45:35 -0500",54,"[-0.031852957, 0.028438816, -0.027885322, -0.0..."


In [ ]:
client = chromadb.Client(Settings()) # Inicializar cliente de Chroma
collection = client.get_or_create_collection("rpp_news") # Crear o conectar una colección (como una “tabla” vectorial)

# Preparar datos para agregar
ids = [str(i) for i in range(len(df_news))]  # IDs únicos
documents = df_news["description"].tolist()
metadatas = df_news[["title", "link", "published"]].astype(str).to_dict(orient="records")
embeddings = df_news["embedding"].tolist()

# Agregar a la colección
collection.add(
    ids=ids,
    documents=documents,
    metadatas=metadatas,
    embeddings=embeddings
)

print(f"Se guardaron {len(embeddings)} noticias en la colección 'rpp_news'")

Se guardaron 50 noticias en la colección 'rpp_news'


In [22]:
def search_news(query: str, model, collection, k: int = 5) -> pd.DataFrame:
    """
    Busca las k noticias más parecidas al texto de consulta (query)
    usando embeddings y similitud.
    """
    query_emb = model.encode([query]) # Crear embedding del texto de búsqueda
    res = collection.query( 
        query_embeddings=query_emb,
        n_results=k
    )
    results = []   # Convertir resultados a DataFrame
    for meta, doc, dist in zip(res["metadatas"][0], res["documents"][0], res["distances"][0]):
        results.append({
            "title": meta.get("title"),
            "description": doc[:200] + "...",   # recorta texto largo
            "link": meta.get("link"),
            "published": meta.get("published"),
            "distance": round(dist, 4)
        })

    df_results = pd.DataFrame(results)
    return df_results

In [23]:
query = "últimas noticias de economía de Peru"
df_results = search_news(query, model, collection, k=5)
df_results

,title,description,link,published,distance
0,¡Tendencia a la baja! Dólar sigue cayendo y ci...,El dólar continúa su tendencia bajista en el P...,https://rpp.pe/economia/economia/dolar-en-peru...,"Mon, 20 Oct 2025 13:55:49 -0500",0.6989
1,Paneles termoaislantes: la solución rápida y e...,Ochmon Perú impulsa la innovación en el sector...,https://rpp.pe/campanas/publirreportaje/panele...,"Mon, 20 Oct 2025 14:04:04 -0500",0.7155
2,¿Seguirá cayendo el dólar o se estabilizará? E...,El dólar acumula una fuerte caída en las últim...,https://rpp.pe/economia/economia/precio-del-do...,"Mon, 20 Oct 2025 13:51:31 -0500",0.8484
3,María Antonieta de las Nieves se despide de la...,La actriz mexicana anunció que dejará de inter...,https://rpp.pe/famosos/celebridades/maria-anto...,"Mon, 20 Oct 2025 15:43:28 -0500",0.8489
4,Precio del dólar Perú y tipo de cambio hoy 20 ...,"En Perú, el tipo de cambio para hoy lunes 20 d...",https://rpp.pe/economia/economia/precio-del-do...,"Mon, 20 Oct 2025 06:05:39 -0500",0.8955


In [ ]:
translator = pipeline("translation", model="facebook/m2m100_418M") # Crear pipeline de traducción español→inglés
translated = [] # Traducir la columna description para tener mejor comprensión
for text in df_news["description"].fillna("").tolist():
    if text.strip() == "":
        translated.append("")
    else:
        result = translator(text, src_lang="es", tgt_lang="en", max_length=512)
        translated.append(result[0]["translation_text"])

df_news["description_en"] = translated # Añadir nueva columna con la traducción
df_news[["description", "description_en"]].head(3)

Device set to use cpu


,description,description_en
0,Universitario sale con todo de local frente a ...,Universitario goes out with everything from lo...
1,Cuál es el ultimo temblor en México y CDMX reg...,What is the last trembling in Mexico and CDMX ...
2,Universitario recibe a Ayacucho FC por la fech...,University receives Ayacucho FC for the 15th d...


In [27]:
df_news.to_csv("../data/rpp_news.csv", index=False, encoding="utf-8")

In [28]:
rpp_texts = df_news["description_en"].fillna("").head(50).tolist() # Tomar las primeras 50 noticias en inglés
llm_sim = pipeline("zero-shot-classification", model="facebook/bart-large-mnli") # Crear el pipeline de clasificación zero-shot
categories = ["World", "Sports", "Business", "Sci/Tech"] # Categorías del dataset AG News
llm_labels = [] # Categorías del dataset AG News
for text in rpp_texts:
    result = llm_sim(text, candidate_labels=categories)
    llm_labels.append(result["labels"][0])  # la categoría con mayor score

df_news["LLM_label"] = llm_labels #  Guardar las etiquetas en el DataFrame
df_news[["description_en", "LLM_label"]].head(5)

Device set to use cpu


,description_en,LLM_label
0,Universitario goes out with everything from lo...,Business
1,What is the last trembling in Mexico and CDMX ...,Sci/Tech
2,University receives Ayacucho FC for the 15th d...,Sports
3,Lula defended Brazil’s firmness against the 50...,Sci/Tech
4,Football Live Channels and TV channels to watc...,Sports


In [34]:
df_llm = df_news[["description_en", "LLM_label"]] # Seleccionar solo las columnas necesarias para el bonus
df_llm.to_csv("../data/rpp_llm_labels.csv", index=False, encoding="utf-8")

In [35]:
df_llm.head(7)

,description_en,LLM_label
0,Universitario goes out with everything from lo...,Business
1,What is the last trembling in Mexico and CDMX ...,Sci/Tech
2,University receives Ayacucho FC for the 15th d...,Sports
3,Lula defended Brazil’s firmness against the 50...,Sci/Tech
4,Football Live Channels and TV channels to watc...,Sports
5,Samantha Batallanos highlighted that her relat...,World
6,Atlético de Madrid wants to win over Arsenal i...,Sports
